In [2]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json
import numpy as np
from json import dumps, loads
import glob

In [3]:
loaded_model = tf.keras.models.load_model(os.path.join('data', "25", 'base.h5'))
json_model = loaded_model.to_json()

2022-12-30 21:24:41.201286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-30 21:24:41.203524: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
with open("model.json", "w") as json_file:
    json_file.write(json_model)

In [5]:
loaded_model.trainable_variables

[<tf.Variable 'lstm/lstm_cell/kernel:0' shape=(25, 320) dtype=float32, numpy=
 array([[-0.0490815 ,  0.03162912,  0.0634328 , ...,  0.1089199 ,
          0.02019161,  0.11620758],
        [-0.11268278,  0.05130196,  0.04136015, ...,  0.0443987 ,
          0.03592196,  0.04857601],
        [ 0.1341333 , -0.15684585, -0.00525478, ..., -0.12209408,
         -0.06746515, -0.01210506],
        ...,
        [-0.08034198, -0.14126386,  0.05556611, ...,  0.0303606 ,
          0.03934618,  0.08981034],
        [ 0.06467141,  0.10404609,  0.05639338, ..., -0.05431012,
          0.03558454,  0.07286088],
        [-0.0499787 , -0.07327107,  0.08109644, ...,  0.07588235,
         -0.09323244,  0.04384843]], dtype=float32)>,
 <tf.Variable 'lstm/lstm_cell/recurrent_kernel:0' shape=(80, 320) dtype=float32, numpy=
 array([[-0.03597874, -0.02925308, -0.06168243, ..., -0.07396556,
         -0.05399349, -0.02789672],
        [-0.00308666,  0.01293187,  0.02346811, ..., -0.00273848,
          0.12792234, -

In [6]:
for i in loaded_model.trainable_weights:
    print(i.shape)

(25, 320)
(80, 320)
(320,)
(80, 320)
(80, 320)
(320,)
(80, 10)
(10,)


In [7]:
for i in loaded_model.trainable_variables:
    print(i.shape)

(25, 320)
(80, 320)
(320,)
(80, 320)
(80, 320)
(320,)
(80, 10)
(10,)


In [8]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()

In [9]:
loaded_model = model_from_json(loaded_model_json)

In [10]:
json_file = open('data/config.json', 'r')
config = json_file.read()

In [11]:
cfg = loads(config)
cfg

{'loss_metric': 'mse',
 'optimizer': 'adam',
 'validation_split': 0.2,
 'dropout': 0.3,
 'lstm_batch_size': 64}

In [12]:
loaded_model.compile(loss=cfg['loss_metric'], optimizer=cfg['optimizer'])

In [13]:
import os
import random
import numpy as np
base_dir = os.path.join("data","25")


base_dir

'data/25'

In [14]:
datasets = [ g for g in glob.glob(base_dir +"/train/*X.npy")]

In [15]:
sel = random.choice(datasets)
x = np.load(sel)
sel = sel.replace("X", "y")
y = np.load(sel)

In [26]:
xs = x[:64, :, :]

In [27]:
ys = y[:64,:]

In [20]:
y.shape

(2558, 10)

In [16]:
data.shape

NameError: name 'data' is not defined

In [31]:
hist = loaded_model.fit(xs,ys,
    batch_size=64,
    epochs=5,
    validation_split=.2
)


Epoch 1/5
1/1 [==============================] - 1s 1s/step - loss: 0.1790 - val_loss: 0.1882
Epoch 2/5
1/1 [==============================] - 1s 1s/step - loss: 0.1767 - val_loss: 0.1901
Epoch 3/5
1/1 [==============================] - 1s 1s/step - loss: 0.1770 - val_loss: 0.1923
Epoch 4/5
1/1 [==============================] - 1s 1s/step - loss: 0.1761 - val_loss: 0.1944
Epoch 5/5
1/1 [==============================] - 1s 1s/step - loss: 0.1751 - val_loss: 0.1964


In [35]:
hist.history

{'loss': [0.17900799214839935,
  0.17669638991355896,
  0.17701032757759094,
  0.1760912388563156,
  0.17509329319000244],
 'val_loss': [0.18817773461341858,
  0.19009192287921906,
  0.1922779679298401,
  0.19443559646606445,
  0.19644783437252045]}

In [41]:
import json
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [50]:
numpyData = {"x": xs}
encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)
print("Printing JSON serialized NumPy array")

Printing JSON serialized NumPy array


In [44]:
print("Decode JSON serialized NumPy array")
decodedArrays = json.loads(encodedNumpyData)

Decode JSON serialized NumPy array


In [48]:
np.array(decodedArrays['array']).shape

(64, 250, 25)

In [49]:
xs.shape

(64, 250, 25)